## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Tuple
import warnings
warnings.filterwarnings("ignore")

## Set Data Paths

In [ ]:
in_path = Path("veh1153_cleaned.csv")

out_localtrend = Path("veh1153_bev_uk_localtrend.csv")
out_lowbound   = Path("veh1153_bev_uk_lowbound.csv")
out_highbound  = Path("veh1153_bev_uk_highbound.csv")

UK_AGG_START = pd.Timestamp("2014-07-31")
PROJECT_START = pd.Timestamp("2011-01-01")  # Jan 2011 month-end

## Read Data

In [37]:
#read csv file
df = pd.read_csv(in_path, index_col = False, low_memory = False)

In [38]:
df.head()

,Column1,Country,Interval,Month,Units,Body Type,Keepership,Petrol,Diesel,HEV Petrol,HEV Diesel,PHEV Petrol,PHEV Diesel,Battery Electric,Range Extended,Fuel Cell,Gas,Others,Total
0,0,England,Annual,2001,Thousands,Buses and coaches,Company,0.1,8.4,0,0,0,0,0,0,0,[low],0,8.6
1,1,England,Annual,2002,Thousands,Buses and coaches,Company,0.1,9.9,0,0,0,0,[low],0,0,[low],[low],10
2,2,England,Annual,2003,Thousands,Buses and coaches,Company,0.1,11,0,0,0,0,[low],0,0,[low],[low],11.1
3,3,England,Annual,2004,Thousands,Buses and coaches,Company,0.1,10.5,0,0,0,0,[low],0,0,[low],[low],10.7
4,4,England,Annual,2005,Thousands,Buses and coaches,Company,[low],10,0,0,0,0,[low],0,0,[low],0,10


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80730 entries, 0 to 80729
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Column1           80730 non-null  int64 
 1   Country           80730 non-null  object
 2   Interval          80730 non-null  object
 3   Month             80730 non-null  object
 4   Units             80730 non-null  object
 5   Body Type         80730 non-null  object
 6   Keepership        80730 non-null  object
 7   Petrol            80730 non-null  object
 8   Diesel            80730 non-null  object
 9   HEV Petrol        80730 non-null  object
 10  HEV Diesel        80730 non-null  object
 11  PHEV Petrol       80730 non-null  object
 12  PHEV Diesel       80730 non-null  object
 13  Battery Electric  80730 non-null  object
 14  Range Extended    80730 non-null  object
 15  Fuel Cell         80730 non-null  object
 16  Gas               80730 non-null  object
 17  Others      

## Helper Functions

In [40]:
# ---------- helpers ----------
def standardise_cols(df: pd.DataFrame) -> pd.DataFrame:
    """Lowercase, strip, and convert spaces to underscores for column names."""
    df = df.copy()
    df.columns = (
        df.columns.str.strip()
        .str.replace(r"\s+", "_", regex=True)
        .str.lower()
    )
    return df

def parse_month_end(s: pd.Series) -> pd.Series:
    """Parse 'Jan-11' strings to pandas month-end timestamps."""
    return pd.to_datetime(s.str.strip(), format="%b-%y") + pd.offsets.MonthEnd(0)

def numeric_from_thousands(s: pd.Series) -> pd.Series:
    """
    Convert values (in thousands) to float; map '[low]' -> NaN so we can impute later.
    We keep 'thousands' until final conversion to units to avoid round-trip noise.
    """
    return pd.to_numeric(s.replace({"[low]": np.nan}), errors="coerce")

def fill_total_thousands(group: pd.DataFrame) -> pd.DataFrame:
    g = group.copy()
    """
    Interpolate 'total_thousands' within each country over time.
    Totals are large and smooth; linear interpolation is acceptable.
    """
    g["total_thousands"] = g["total_thousands"].interpolate(method="linear", limit_direction="both")
    return g

def local_trend_impute_units(series_units: pd.Series, mask_impute: pd.Series,
                             lower:int=1, upper:int=9) -> pd.Series:
    """
    Impute NaNs in a *units* series using a short, centered local mean of neighbours.
    - Try windows of ±1, ±2, ±3 (and ±6 as a last resort)
    - Round to nearest integer and clip to [lower, upper] (ONS SDC N=10 => suppressed <10)
    """
    s = series_units.copy()
    idxs = s.index[mask_impute]
    # Pre-compute a list of all positions for quick lookup
    for ix in idxs:
        # try windows of size 1, 2, 3 (i.e., +/-1, +/-2, +/-3 months)
        est = None
        for w in (1, 2, 3, 6):  # allow broader window as last resort
            left = s.loc[:ix].iloc[:-1].tail(w)     # previous w non-target points
            right = s.loc[ix:].iloc[1:w+1]          # next w points
            vals = pd.concat([left, right]).dropna()
            if len(vals) > 0:
                est = float(vals.mean())
                break
        if est is None or np.isnan(est):
            # fallback: nearest non-missing backwards then forwards
            prev = s.loc[:ix].iloc[:-1].dropna()
            nxt  = s.loc[ix:].iloc[1:].dropna()
            if len(prev) > 0:
                est = float(prev.iloc[-1])
            elif len(nxt) > 0:
                est = float(nxt.iloc[0])
            else:
                est = float(lower)
        # round to nearest integer and clip to [1,9]
        est_int = int(np.rint(est))
        est_int = max(lower, min(upper, est_int))
        s.loc[ix] = est_int
    return s

def build_uk_series(df_country_units: pd.DataFrame) -> pd.DataFrame:
    devolved = ["England", "Scotland", "Wales", "Northern Ireland"]

    early = df_country_units[
        (df_country_units["date"] < UK_AGG_START) &
        (df_country_units["country"].isin(devolved))
    ]
    early_agg = (
        early.groupby("date")
        .agg(
            bev_units=("bev_units", "sum"),
            total_cars=("total_units", "sum"),
            low_flag=("low_flag_bev", "max"),
        )
        .assign(source_flag="Sum_devolved")
    )

    late = df_country_units[
        (df_country_units["date"] >= UK_AGG_START) &
        (df_country_units["country"] == "United Kingdom")
    ]
    late_agg = (
        late.groupby("date")
        .agg(
            bev_units=("bev_units", "sum"),
            total_cars=("total_units", "sum"),
            low_flag=("low_flag_bev", "max"),
        )
        .assign(source_flag="UK_total")
    )

    uk = pd.concat([early_agg, late_agg]).sort_index()

    # restrict to project window starting Jan 2011
    uk = uk[uk.index >= PROJECT_START]

    uk["bev_share"] = uk["bev_units"] / uk["total_cars"]
    uk["bev_units_log"] = np.where(uk["bev_units"] > 0, np.log(uk["bev_units"]), np.nan)
    uk = uk[["bev_units", "total_cars", "bev_share", "bev_units_log", "source_flag", "low_flag"]]
    return uk

## Data Filtering

In [41]:
raw = pd.read_csv(in_path, index_col = False, low_memory = False)
df = standardise_cols(raw)

required_cols = {"interval","body_type","keepership","country","month","battery_electric","total"}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing expected columns: {missing}")

# Filter to monthly, cars, keepership total
df = df[(df["interval"]=="Monthly") & (df["body_type"]=="Cars") & (df["keepership"]=="Total")].copy()

# Parse date
df["date"] = parse_month_end(df["month"])
df["country"] = df["country"].str.strip()

# limit to >= Jan 2011 in the raw, so downstream groupbys don't carry earlier noise
df = df[df["date"] >= PROJECT_START].copy()

# Create flags and numeric thousands
df["low_flag_bev"] = (df["battery_electric"]=="[low]").astype(int)
df["bev_thousands"] = numeric_from_thousands(df["battery_electric"])
df["total_thousands"] = numeric_from_thousands(df["total"])

# Fill totals within country (linear interp)
df = (df.sort_values(["country","date"])
        .groupby("country", group_keys=False)
        .apply(fill_total_thousands))

In [42]:
df.head()

,column1,country,interval,month,units,body_type,keepership,petrol,diesel,hev_petrol,...,battery_electric,range_extended,fuel_cell,gas,others,total,date,low_flag_bev,bev_thousands,total_thousands
2230,2230,England,Monthly,Jan-11,Thousands,Cars,Total,53.2,55.1,1.5,...,[low],0,0,[low],[low],109.9,2011-01-31,1,NaN,109.9
42595,42595,England,Monthly,Jan-11,Percentage of total,Cars,Total,48.4,50.2,1.4,...,[low],0,0,[low],[low],100,2011-01-31,1,NaN,100.0
2231,2231,England,Monthly,Feb-11,Thousands,Cars,Total,26.8,26.9,0.5,...,[low],0,0,[low],[low],54.2,2011-02-28,1,NaN,54.2
42596,42596,England,Monthly,Feb-11,Percentage of total,Cars,Total,49.4,49.6,1,...,0.1,0,0,[low],[low],100,2011-02-28,0,0.1,100.0
2232,2232,England,Monthly,Mar-11,Thousands,Cars,Total,158.6,151.8,4.5,...,0.2,0,0,[low],0,315.2,2011-03-31,0,0.2,315.2


In [43]:
df.describe()

,column1,date,low_flag_bev,bev_thousands,total_thousands
count,1776.000000,1776,1776.000000,1485.000000,1776.000000
mean,40679.135135,2018-04-05 19:24:19.459459328,0.163851,4.907407,95.038795
min,2230.000000,2011-01-31 00:00:00,0.000000,0.000000,0.100000
25%,18911.750000,2015-04-22 12:00:00,0.000000,0.300000,43.675000
50%,40211.000000,2018-05-15 12:00:00,0.000000,0.700000,100.000000
75%,59276.250000,2021-06-07 12:00:00,0.000000,7.200000,100.000000
max,78192.000000,2024-06-30 00:00:00,1.000000,48.500000,565.600000
std,23673.899174,NaN,0.370245,8.002281,76.165759


## Create datasets with [low] values imputed with interpolated, low bound and upper bound values

In [45]:
# Scenario A: Local-trend 1..9
dA = df.copy()
dA["bev_units"] = (dA["bev_thousands"] * 1000).round()
dA.loc[dA["low_flag_bev"] == 1, "bev_units"] = np.nan
dA["total_units"] = (dA["total_thousands"] * 1000).round().astype("Int64")

filled_chunks = []
for c, g in dA.groupby("country"):
    g = g.set_index("date").sort_index()
    s = g["bev_units"].astype(float)
    mask = s.isna()
    s_filled = local_trend_impute_units(s, mask, lower=1, upper=9)
    g["bev_units"] = s_filled.astype(int)
    filled_chunks.append(g.reset_index())
dA2 = pd.concat(filled_chunks, ignore_index=True)
uk_localtrend = build_uk_series(dA2[["date", "country", "bev_units", "total_units", "low_flag_bev"]])

# Scenario B: Low-bound = 1
dB = df.copy()
dB["bev_units"] = (dB["bev_thousands"] * 1000).round().astype(float)
dB.loc[dB["low_flag_bev"] == 1, "bev_units"] = 1.0
dB["bev_units"] = dB["bev_units"].round().astype(int)
dB["total_units"] = (dB["total_thousands"] * 1000).round().astype("Int64")
uk_lowbound = build_uk_series(dB[["date", "country", "bev_units", "total_units", "low_flag_bev"]])

# Scenario C: High-bound = 9
dC = df.copy()
dC["bev_units"] = (dC["bev_thousands"] * 1000).round().astype(float)
dC.loc[dC["low_flag_bev"] == 1, "bev_units"] = 9.0
dC["bev_units"] = dC["bev_units"].round().astype(int)
dC["total_units"] = (dC["total_thousands"] * 1000).round().astype("Int64")
uk_highbound = build_uk_series(dC[["date", "country", "bev_units", "total_units", "low_flag_bev"]])

# Save
uk_localtrend.to_csv(out_localtrend, index_label="date")
uk_lowbound.to_csv(out_lowbound, index_label="date")
uk_highbound.to_csv(out_highbound, index_label="date")

out_localtrend,out_lowbound, out_highbound

(WindowsPath('veh1153_bev_uk_localtrend.csv'),
 WindowsPath('veh1153_bev_uk_lowbound.csv'),
 WindowsPath('veh1153_bev_uk_highbound.csv'))

In [46]:
uk_localtrend.tail()

,bev_units,total_cars,bev_share,bev_units_log,source_flag,low_flag
date,,,,,,
2024-02-29,31800,189300,0.167987,10.367222,UK_total,0
2024-03-31,63500,422600,0.15026,11.058795,UK_total,0
2024-04-30,39200,237600,0.164983,10.576432,UK_total,0
2024-05-31,43100,252100,0.170964,10.671278,UK_total,0
2024-06-30,52800,282700,0.18677,10.874266,UK_total,0


In [47]:
uk_localtrend.index.min

<bound method Index.min of DatetimeIndex(['2011-01-31', '2011-02-28', '2011-03-31', '2011-04-30',
               '2011-05-31', '2011-06-30', '2011-07-31', '2011-08-31',
               '2011-09-30', '2011-10-31',
               ...
               '2023-09-30', '2023-10-31', '2023-11-30', '2023-12-31',
               '2024-01-31', '2024-02-29', '2024-03-31', '2024-04-30',
               '2024-05-31', '2024-06-30'],
              dtype='datetime64[ns]', name='date', length=162, freq=None)>

In [48]:
uk_localtrend.describe()

,bev_units,total_cars,bev_share,bev_units_log,low_flag
count,162.000000,162.0,162.0,162.000000,162.000000
mean,11912.543210,332475.308642,0.048164,7.783561,0.259259
std,17540.735898,134772.859323,0.071419,2.117134,0.439587
min,18.000000,104200.0,0.00005,2.890372,0.000000
25%,600.000000,244250.0,0.001826,6.396930,0.000000
50%,1700.000000,281550.0,0.005859,7.438384,0.000000
75%,19550.000000,431800.0,0.085783,9.880481,1.000000
max,74600.000000,759600.0,0.323224,11.219896,1.000000


In [2]:
rhdi = pd.read_csv('rhdi_monthly_stepfill_2011_latest.csv')
rhdi.head()

,date,rhdi_sa
0,2011-01-31 23:59:59.999999999,88.303735
1,2011-02-28 23:59:59.999999999,88.303735
2,2011-03-31 23:59:59.999999999,88.303735
3,2011-04-30 23:59:59.999999999,89.161053
4,2011-05-31 23:59:59.999999999,89.161053


In [3]:
rhdi.tail()

,date,rhdi_sa
163,2024-08-31 23:59:59.999999999,104.47030
164,2024-09-30 23:59:59.999999999,104.47030
165,2024-10-31 23:59:59.999999999,106.30741
166,2024-11-30 23:59:59.999999999,106.30741
167,2024-12-31 23:59:59.999999999,106.30741
